In [1]:
!nvidia-smi

Fri Aug 12 15:29:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.67       Driver Version: 451.67       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 207... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   47C    P8    15W /  N/A |   1047MiB /  8192MiB |     10%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [2]:
!nvcc --version

'nvcc'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\ai_15\appdata\local\temp\pip-req-build-4pi06zxc
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4320 sha256=b2eeecc5585478f91ebe900ec65e909740e1f6cdb06e46a3f97e0b79babcdc80
  Stored in directory: C:\Users\AI_15\AppData\Local\Temp\pip-ephem-wheel-cache-i_r7x6x6\wheels\f3\08\cc\e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\AI_15\AppData\Local\Temp\pip-req-build-4pi06zxc'


In [4]:
%load_ext nvcc_plugin

directory c:\Users\AI_15\momo\python_prj\2022_AI_Expert\GPU_CUDA\src already exists
Out bin c:\Users\AI_15\momo\python_prj\2022_AI_Expert\GPU_CUDA\result.out


In [5]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define BLOCK_SIZE 16

/*************************************************
Function name: gpu_matrix_mult

Parameters:
            &a GPU device pointer to a m X n matrix (A)
            &b GPU device pointer to a n X k matrix (B)
            &c GPU device output pointer to a m X k matrix (C)

Note:
      grid and block should be configured as:
            dim3 dimGrid((k + BLOCKSIZE - 1) / BLOCK_SIZE, (m + BLOCK_SIZE - 1) / BLOCK_SIZE);
            dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
*************************************************/

__global__ void gpu_matrix_mult(int *a, int *b, int *c, int m, int n, int k)
{
    /*
    Part 1. Write GPU kernel code here for executing matrix multiplication
    Hint: Column index is calculated as, blockIdx.x * blockDim.x + threadIdx.x;

    
    Part 1 ends here
    */
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum;
 
    if(col < k && row < m)
    {
        sum=0;
        for(int i=0; i < n; i++)
        {
            sum += a[row*n + i] * b[i * k + col];
        }
        c[row*k + col] = sum;
    }
}

/*************************************************
Function name: gpu_tiled_matrix_mult

Parameters:
            &a GPU device pointer to a n X n matrix (A)
            &b GPU device pointer to a n X n matrix (B)
            &c GPU device output pointer to a n X n matrix (C)

Note:
      grid and block should be configured as:
            dim3 dimGrid((k + BLOCKSIZE - 1) / BLOCK_SIZE, (m + BLOCK_SIZE - 1) / BLOCK_SIZE);
            dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
*************************************************/

__global__ void gpu_tiled_matrix_mult(int *d_a, int *d_b, int *d_result, int n)
{
    /*
    Part 2. Write GPU kernel code for executing tiled matrix multiplication
    Hint: Need __syncthreads() for the correct results


    Part 2 ends here
    */
    int k = 32;
    
    int tile_A = blockIdx.y * blockDim.y;
    int tile_B = blockIdx.x * blockDim.x;
 
    int row = tile_A + threadIdx.y;
    int col = tile_B + threadIdx.x;
 
    __shared__ int aTile[32][32];
    __shared__ int bTile[32][32];
    aTile[threadIdx.y][threadIdx.x] = d_a[row * k + threadIdx.x];
    bTile[threadIdx.y][threadIdx.x] = d_b[threadIdx.y * n + col];
    __syncthreads();
 
    int tile_element_sum = 0;
    
    for (int i = 0; i < k; i++) {
        tile_element_sum += aTile[threadIdx.y][i] * bTile[i][threadIdx.x];
    }

    d_result[row * n + col] = tile_element_sum;
}

/*************************************************
Function name: cpu_matrix_mult

Parameters:
            &a CPU host pointer to a n X n matrix (A)
            &b CPU host pointer to a n X n matrix (B)
            &c CPU host output pointer to a n X n matrix (C)
*************************************************/

__host__ void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i)
    {
        for (int j = 0; j < k; ++j)
        {
            int tmp = 0.0;
            for (int h = 0; h < n; ++h)
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}

/*************************************************
Function name: main

Test and Compare
*************************************************/

int main(int argc, char const *argv[])
{
    int m, n, k;
    srand(time(0));
    
    //Set the size of matrices
    m = 256;
    n = 256;
    k = 256;

    /*
    Part 3-1. Allocate memory in host DRAM, h_cc is used to store the CPU result

    int *h_a, *h_b, *h_c, *h_cc;
    

    Part 3-1 ends here
    */
    int *h_a, *h_b, *h_c, *h_cc;
    
    cudaMallocHost((void **) &h_a, sizeof(int) * m * n);
    cudaMallocHost((void **) &h_b, sizeof(int) * n * k);
    cudaMallocHost((void **) &h_c, sizeof(int) * m * k);
    cudaMallocHost((void **) &h_cc, sizeof(int) * m * k);
 
    int i, j;
    // Random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] = rand() % 1024;
        }
    }

    // Random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = rand() % 1024;
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // Events to measure the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Start to measure the execution time of GPU version
    cudaEventRecord(start, 0);

    /*
    Part 3-2. Allocate memory space on the device (GPU)
              & Copy matrix A and B from host to device memory
    
    int *d_a, *d_b, *d_c;


    Part 3-2 ends here
    */

    int *d_a, *d_b, *d_c;
    
    cudaMalloc((void **) &d_a, sizeof(int) * m * n);
    cudaMalloc((void **) &d_b, sizeof(int) * n * k);
    cudaMalloc((void **) &d_c, sizeof(int) * m * k);
 
    cudaMemcpy(d_a, h_a, sizeof(int) * m * n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int) * n * k, cudaMemcpyHostToDevice);
 
    unsigned int grid_rows = (m + BLOCK_SIZE - 1) / BLOCK_SIZE;
    unsigned int grid_cols = (k + BLOCK_SIZE - 1) / BLOCK_SIZE;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);

    /*
    Part 3-3. Launch GPU kernel
              & Transfer results from the device to host
    Note:
          For the tiled multiplication with square matrices (i.e., m = n = k), launch gpu_square_matrix_mult
          Otherwise, launch regular matrix multiplication kernel
    
    if (m == n && n == k && n > BLOCK_SIZE && (n % BLOCK_SIZE == 0)) {
        Launch the tiled matrix multiplication kernel
    }
    else {
        Launch the normal matrix multplication kernel
    }
    
    // Transfer the results from device to host
    
    Part 3-3 ends here
    */
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, m, n, k);
    //gpu_tiled_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, m, n, k);
    cudaMemcpy(h_c, d_c, sizeof(int) * m * k, cudaMemcpyDeviceToHost);

    cudaDeviceSynchronize();
    
    // Time counting terminate
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // GPU computing time elapse
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("\n\nGPU execution time on matrix multiplication of %dx%d . %dx%d: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    // CPU version
    cudaEventRecord(start, 0);

    cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    printf("CPU execution time on matrix multiplication of %dx%d . %dx%d: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);

    // Validate the results computed by GPU
    int all_ok = 1;
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < k; ++j) {
            // Uncomment below to see the actual results on both CPU and GPU
            // printf("CPU result [%d][%d]:%d == GPU result [%d][%d]:%d, ", i, j, h_cc[i * k + j], i, j, h_c[i * k + j]);
            if (h_cc[i * k + j] != h_c[i * k + j]) {
                all_ok = 0;
            }
        }
        // printf("\n");
    }

    // Compute the speedup
    if (all_ok) {
        printf("All results are correct !!!, speedup = %f\n", cpu_elapsed_time_ms / gpu_elapsed_time_ms);
    }
    else {
        printf("Incorrect results\n");
    }

    /*
    Part 3-4. Free the device and host memory


    Part 3-4 ends here
    */
    cudaFree(d_a);
    cudaFree(d_a);
    cudaFree(d_c);
    cudaFreeHost(h_a);
    cudaFreeHost(h_a);
    cudaFreeHost(h_c);
    cudaFreeHost(h_cc);
 
    return 0;
}

In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <cstdlib>
#include <time.h>
#include <iostream>
#include <string>
#include <math.h>
#include <assert.h>

#define BLOCK_SIZE 16

using namespace std;


__global__ void Convolution(float* A, float* B, float* C, int numARows, int numACols, int numBRows, int numBCols, int numCRows, int numCCols)
{
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;
    float sum;
   
    //A * B = C
    if(row<numCRows&&col<numCCols){
        sum =0;

		/*Part 1. write GPU kernel code for executing convolution
		**convolution with the entire filter becomes the one element of the matrix C
		***row/col indicates the index of output matrix C




		Part 1 ends
		*/
		for(size_t i=0; i<numBCols; i++){
			for(size_t j=0; j<numBRows; j++){
				  sum += A[row*numACols+i*numACols+col+j]*B[i*numBCols+j];
			}	  
		}
		C[row*numCCols+col]=sum;	
    
    }    

}

__host__ void cpu_Convolution(float *A, float *B, float *C, int asize, int bsize){

	int csize=asize-bsize+1;
	int sum;
	for(int i=0;i<csize;i++){
		
		for (int j=0;j<csize;j++){
			sum=0;
			for(int k=0;k<bsize;k++){
				
				for(int l=0;l<bsize;l++){
					sum+=A[i*asize+k*asize+j+l]*B[k*bsize+l];

				}
				
			}
			C[i*csize+j]=sum;
		}



	}
	




}

void randomInit(float* data, int size)
{
	for (int i = 0; i < size; ++i)
		data[i] = rand() %10;
}

int main()
{
	srand(time(0));
	int a, b, c;
	cudaEvent_t start_G, stop_G;
	float gpu_miliseconds, cpu_miliseconds;
	cudaEventCreate(&start_G);
	cudaEventCreate(&stop_G);
	a=5; 	//size of the feature map 
	b=3;	//size of the filter
    c=a-b+1;

	/*Part 2-1.	initilize matrix and filter
	when the input and filter size are a and b, what is the output size c?
	c= ??; 

	Part 2-1 ends
	*/ 
	unsigned int size_A = a * a;
	unsigned int size_B = b * b;
	unsigned int size_C = c * c;
	
	/*Part 2-2. initialize matrix and filter
	allocate memory in host DRAM
	float *h_A, *h_B, *h_C, *h_C_cpu;

	*/
	unsigned int mem_size_A = sizeof(float) * size_A;
	float* h_A = (float*)malloc(mem_size_A);
	unsigned int mem_size_B = sizeof(float) * size_B;
	float* h_B = (float*)malloc(mem_size_B);
	unsigned int mem_size_C = sizeof(float) * size_C;
	float* h_C = (float*)malloc(mem_size_C);
	float* h_C_cpu = (float*)malloc(mem_size_C);

	randomInit(h_A, size_A);

	for (int i = 0; i < size_B; ++i)
    {
        h_B[i] = rand() %4;
    }

	/*Part 3. allocate device memory and memory copy from the host
	allocate memory space on GPU, then copy matrix A and B from the host to GPU
	float *d_A, *d_B, *d_C;		//h_A, h_B -> d_A, d_B

	Part 3 ends
	*/
	float *d_A, *d_B, *d_C;		//h_A, h_B -> d_A, d_B

	cudaMalloc((void**)&d_A, mem_size_A);
	cudaMalloc((void**)&d_B, mem_size_B);
	cudaMalloc((void**)&d_C, mem_size_C);

	cudaMemcpy(d_A, h_A, mem_size_A, cudaMemcpyHostToDevice);
	cudaMemcpy(d_B, h_B, mem_size_B, cudaMemcpyHostToDevice);

	unsigned int grid_rows= (c+BLOCK_SIZE-1) / BLOCK_SIZE;
	unsigned int grid_cols= (c+BLOCK_SIZE-1) / BLOCK_SIZE;
	
	dim3 dimGrid(grid_cols,grid_rows);	
	dim3 dimBlock(BLOCK_SIZE,BLOCK_SIZE);
	
	cudaEventRecord(start_G,0);

	/*Part 4-1. launch GPU kerenl
	launch GPU kernel and execute convolution between A and B, providing values for C
	
	

	Part 4-1 ends
	*/

	Convolution<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, a, a, b, b, c, c);

	cudaEventRecord(stop_G,0);

	cudaEventSynchronize(stop_G);
	/*Part 4-2. data copy from device to host
	transfer the calculated data from gpu to host ( d_C -> h_C)

	*/
	cudaEventRecord(stop_G,0);
 
	cudaEventSynchronize(stop_G);

	cudaMemcpy(h_C, d_C, mem_size_C, cudaMemcpyDeviceToHost);

	cudaEventElapsedTime(&gpu_miliseconds, start_G, stop_G);

	printf("\nTime took to compute matrix A(%d x %d) with filter B(%d x %d) on GPU is %f ms  \n \n", a, a,b,b, gpu_miliseconds);
    printf("matrix A\n");
	for (int i = 0;i < a;i++)
	{
		for (int j = 0;j < a;j++)
		{
			printf("%f\t", h_A[i*a + j]);
		}
		printf("\n");
	}printf("\n");
    printf("matrix B\n");
    for (int i = 0;i < b;i++)
	{
		for (int j = 0;j < b;j++)
		{
			printf("%f\t", h_B[i*b + j]);
		}
		printf("\n");
	}printf("\n");
    printf("matrix C\n");
	for (int i = 0;i < c;i++)
	{
		for (int j = 0;j < c;j++)
		{
			printf("%f\t", h_C[i*c + j]);
		}
		printf("\n");
	}
	cudaEventRecord(start_G, 0);
	cpu_Convolution(h_A, h_B, h_C_cpu, a,b);	// convolution execution with CPU
	cudaEventRecord(stop_G,0);
	cudaEventSynchronize(stop_G);
	cudaEventElapsedTime(&cpu_miliseconds, start_G, stop_G);
	printf("\nTime took to compute matrix A(%d x %d) with filter B(%d x %x) on CPU is %f ms  \n \n", a, a,b,b, cpu_miliseconds);
    
	for (int i = 0;i < c;i++)
	{
		for (int j = 0;j < c;j++)
		{
			printf("%f\t", h_C_cpu[i*c + j]);
		}
		printf("\n");
	}

	free(h_A);
	free(h_B);
	free(h_C);
	free(h_C_cpu);
	cudaFree(d_A);
	cudaFree(d_B);
	cudaFree(d_C);

	return EXIT_SUCCESS;
}
